# Exploratory Data Analysis - Bengali AI Dataset
# Grapheme Combinations

This dataset contains images of individual hand-written [Bengali characters](https://en.wikipedia.org/wiki/Bengali_alphabet). 
Bengali characters (graphemes) are written by combining three components: a grapheme_root
, vowel_diacritic, and consonant_diacritic. Your challenge is to classify the components of the grapheme in each
image. There are roughly 10,000 possible graphemes, of which roughly 1,000 are represented in the training set. The
test set includes some graphemes that do not exist in train but has no new grapheme components. It takes a lot of
volunteers filling out [sheets like this](https://github.com/BengaliAI/graphemePrepare/blob/master/collection/A4/form_1.jpg)
to generate a useful amount of real data; focusing the problem on the grapheme components rather than on recognizing
whole graphemes should make it possible to assemble a Bengali OCR system without handwriting samples for all 10,000
graphemes.

In [1]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.metrics import confusion_matrix
from IPython.display import Markdown, HTML
# from ..code.jupyter import grid_df_display, combination_matrix

pd.set_option('display.max_columns',   500)
pd.set_option('display.max_colwidth',   -1 if os.environ.get('KAGGLE_KERNEL_RUN_TYPE','') else None)

%load_ext autoreload
%autoreload 2

## Custom Library Functions

In [2]:
# Source: https://stackoverflow.com/questions/38783027/jupyter-notebook-display-two-pandas-tables-side-by-side/50899244#50899244
import pandas as pd
from IPython.display import display,HTML

def grid_df_display(list_dfs, rows = 2, cols=3, fill = 'cols'):
    if fill not in ['rows', 'cols']: print("grid_df_display() - fill must be one of: 'rows', 'cols'")

    html_table = "<table style='width:100%; border:0px'>{content}</table>"
    html_row   = "<tr style='border:0px'>{content}</tr>"
    html_cell  = "<td style='width:{width}%;vertical-align:top;border:0px'>{{content}}</td>"
    html_cell  = html_cell.format(width=100/cols)

    cells = [ html_cell.format(content=df.to_html()) for df in list_dfs[:rows*cols] ]
    cells += cols * [html_cell.format(content="")] # pad

    if fill == 'rows':   # fill in rows first (first row: 0,1,2,... col-1)
        grid = [ html_row.format(content="".join(cells[i:i+cols])) for i in range(0,rows*cols,cols)]
    elif fill == 'cols': # fill columns first (first column: 0,1,2,..., rows-1)
        grid = [ html_row.format(content="".join(cells[i:rows*cols:rows])) for i in range(0,rows)]
    else:
        grid = []

    # noinspection PyTypeChecker
    display(HTML(html_table.format(content="".join(grid))))

    # add extra dfs to bottom
    [display(list_dfs[i]) for i in range(rows*cols,len(list_dfs))]


if __name__ == "main":
    list_dfs = []
    list_dfs.extend((pd.DataFrame(2*[{"x":"hello"}]),
                     pd.DataFrame(2*[{"x":"world"}]),
                     pd.DataFrame(2*[{"x":"gdbye"}])))

    grid_df_display(3*list_dfs)

In [3]:
from typing import Union

import pandas as pd
from pandas.io.formats.style import Styler


def combination_matrix(dataset: pd.DataFrame, x: str, y: str, z: str,
                       format=None, unique=True) -> Union[pd.DataFrame, Styler]:
    """
    Returns a combination matrix, showing all valid combinations between three DataFrame columns.
    Sort of like a heatmap, but returning lists of (optionally) unique values

    :param dataset: The dataframe to create a combination_matrx from
    :param x: column name to use for the X axis
    :param y: column name to use for the Y axis
    :param z: column name to use for the Z axis (values that appear in the cells)
    :param format: '', ', '-', ', '\n'    = format value lists as "".join() string
                    str, bool, int, float = cast value lists
    :param unique:  whether to return only unique values or not - eg: combination_matrix(unique=False).applymap(sum)
    :return: returns nothing
    """
    unique_y = sorted(dataset[y].unique())
    combinations = pd.DataFrame({
        n: dataset.where(lambda df: df[y] == n)
            .groupby(x)[z]
            .pipe(lambda df: df.unique() if unique else df )
            .apply(list)
            .apply(sorted)
        for n in unique_y
    }).T

    if isinstance(format, str):
        combinations = combinations.applymap(
            lambda cell: f"{format}".join([str(value) for value in list(cell) ])
            if isinstance(cell, list) else cell
        )
    if format == str:   combinations = combinations.applymap(lambda cell: str(cell)      if isinstance(cell, list) and len(cell) > 0 else ''     )
    if format == bool:  combinations = combinations.applymap(lambda cell: True           if isinstance(cell, list) and len(cell) > 0 else False  )
    if format == int:   combinations = combinations.applymap(lambda cell: int(cell[0])   if isinstance(cell, list) and len(cell)     else ''     )
    if format == float: combinations = combinations.applymap(lambda cell: float(cell[0]) if isinstance(cell, list) and len(cell)     else ''     )

    combinations.index.rename(y, inplace=True)
    combinations.fillna('', inplace=True)
    if format == '\n':
        return combinations.style.set_properties(**{'white-space': 'pre-wrap'})  # needed for display
    else:
        return combinations  # Allows for subsequent .applymap()

## Inspect Raw Data

In [4]:
!ls ../input/bengaliai-cv19/

class_map.csv		   test_image_data_3.parquet
sample_submission.csv	   train.csv
test.csv		   train_image_data_0.parquet
test_image_data_0.parquet  train_image_data_1.parquet
test_image_data_1.parquet  train_image_data_2.parquet
test_image_data_2.parquet  train_image_data_3.parquet


In [5]:
dataset = pd.read_csv('../input/bengaliai-cv19/train.csv'); 
# for key in ['grapheme_root','vowel_diacritic','consonant_diacritic','grapheme']:
#     dataset[key] = dataset[key].astype('category')  # ensures groupby().count() shows zeros
dataset.head()

,image_id,grapheme_root,vowel_diacritic,consonant_diacritic,grapheme
0,Train_0,15,9,5,ক্ট্রো
1,Train_1,159,0,0,হ
2,Train_2,22,3,5,খ্রী
3,Train_3,53,2,2,র্টি
4,Train_4,71,9,5,থ্রো


## Question: How many unique graphemes are there?

There are 168 grapheme roots, 11 vowel diacritics, 7 consonant diacritics, and 1295 unique graphemes within the 20k training dataset. 

In [6]:
unique = dataset.apply(lambda col: col.nunique()); unique

image_id               200840
grapheme_root             168
vowel_diacritic            11
consonant_diacritic         7
grapheme                 1295
dtype: int64

## Question: Can all diacritics be used with any grapheme?

- Documentation claims 10,000+ possible graphemes, which is indeed `168 * 11 * 7 = 12936`

- Assuming that the training dataset is representative of common usage, 
  then certian combinations may never (or rarely) be used in practice).

- Unconfirmed Theory: the physics of the human mouth may make such combinations unpronouncable.

- Conclusion: it may be able infer excluded combinations using simple logical rules

### Vowel / Consonant Combinations:
- Vowel #0 and Consonant #0 combine with everything
- Vowels #3, #5, #6, #8 have limited combinations with Consonants 
- Consonant #3 is never combined except with Vowel #0
- Consonant #6 only combineds with Vowels #0 and #1

In [7]:
combination_matrix(dataset, x='consonant_diacritic', y='vowel_diacritic', z='consonant_diacritic', unique=False).applymap(len)

consonant_diacritic,0.0,1.0,2.0,3.0,4.0,5.0,6.0
vowel_diacritic,,,,,,,
0,23960,768,6262,619,5413,4180,306
1,18799,2843,3838,0,6573,3752,1081
2,17449,464,3764,0,1255,3035,0
3,11391,0,2290,0,0,2471,0
4,11832,1215,1563,0,2206,2032,0
5,3794,0,297,0,784,422,0
6,3873,0,463,0,0,0,0
7,16991,1197,3778,0,4072,2685,0
8,3210,0,0,0,167,151,0


### Grapheme Root Combinations:
- Vowel #0 and Consonant #0 combine with (nearly) everything
- ALL Roots combine with some Consonant #0
- Several Roots do NOT combine with Vowel #0 = [26, 28, 33, 34, 73, 82, 108, 114, 126, 152, 157, 158, 163]
- Several Roots do combine ALL Vowels = [13, 23, 64, 72, 79, 81, 96, 107, 113, 115, 133, 147]}
- Only Root #107 combines with ALL Consonants

In [8]:
root_vowels            = dataset.groupby('grapheme_root')['vowel_diacritic'].unique().apply(sorted).to_frame().T
root_consonants        = dataset.groupby('grapheme_root')['consonant_diacritic'].unique().apply(sorted).to_frame().T
root_vowels_values     = root_vowels.applymap(len).values.flatten()
root_consonants_values = root_consonants.applymap(len).values.flatten()

display(root_vowels)
display({
    "mean":   root_vowels_values.mean(),
    "median": np.median( root_vowels_values ),
    "min":    root_vowels_values.min(),
    "max":    root_vowels_values.max(),
    "unique_vowels":    unique['vowel_diacritic'],
    "root_combine_0":   sum([ 0 in lst for lst in root_vowels.values.flatten() ]),
    "unique_roots":     unique['grapheme_root'],
    "root_combine_not_0": str([ index for index, lst in enumerate(root_vowels.values.flatten()) if 0 not in lst ]),    
    "root_combine_all":       [ index for index, lst in enumerate(root_vowels.values.flatten()) if len(lst) == unique['vowel_diacritic'] ],
})
# print('--------------------')
display(root_consonants)
display({
    "mean":   root_consonants_values.mean(),
    "median": np.median( root_consonants_values ),
    "min":    root_consonants_values.min(),
    "max":    root_consonants_values.max(),
    "unique_consonants":  unique['consonant_diacritic'],
    "root_combine_0": sum([ 0 in lst for lst in root_consonants.values.flatten() ]),
    "unique_roots":   unique['grapheme_root'],
    "root_combine_not_0": str([ index for index, lst in enumerate(root_consonants.values.flatten()) if 0 not in lst ]),        
    "root_combine_all":       [ index for index, lst in enumerate(root_consonants.values.flatten()) if len(lst) == unique['consonant_diacritic'] ],
})

grapheme_root,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167
vowel_diacritic,[0],[0],"[0, 1]",[0],[0],[0],[0],[0],[0],"[0, 1]",[0],[0],[0],"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]","[0, 1, 2, 7, 9]","[0, 1, 2, 7, 9]","[0, 1, 2, 6, 7, 9]","[0, 1, 2, 7, 9]","[0, 1, 2, 3, 4, 7, 9, 10]","[0, 10]","[0, 3]","[0, 1, 2, 3, 7]","[0, 1, 2, 3, 4, 6, 7, 8, 9]","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]","[0, 1]","[0, 1, 2, 3, 4, 7]",[7],"[0, 2]","[1, 2, 4, 9]","[0, 1, 2, 4, 5, 6, 7, 9]","[0, 2, 7]","[0, 1, 2, 4, 7]","[0, 1, 2, 3, 4, 6, 7, 9]",[2],"[1, 2]","[0, 1, 2]","[0, 1, 2, 3, 4, 7, 9]","[0, 2]","[0, 1, 2, 3, 4, 5, 7, 8, 9, 10]","[0, 1, 2, 4]","[0, 1, 2, 4, 7]","[0, 1]","[0, 1, 2, 4, 7, 9]","[0, 1, 2, 3, 4, 7, 8, 9]","[0, 1, 2, 3, 4, 7, 9]",[0],"[0, 1, 3, 7]","[0, 1, 2]","[0, 1, 2, 4, 7, 9]","[0, 1]","[0, 1, 2, 4, 7]","[0, 2]","[0, 1, 2, 3, 4, 7]","[0, 1, 2, 3, 4, 5, 7, 9]","[0, 1, 2, 9]","[0, 1, 2, 3, 4, 7, 9]","[0, 1, 2, 3, 4, 7, 9]","[0, 1, 2, 3]","[0, 1, 2, 4, 7, 9, 10]","[0, 1, 2, 3, 4, 7, 9]","[0, 1, 4]","[0, 1, 2, 7]","[0, 1, 2, 3, 4, 7]",[0],"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]","[0, 1, 2, 3, 7, 9]","[0, 1, 2, 3, 7]","[0, 1, 7]","[0, 2, 3, 7]","[0, 1, 2, 3, 7]","[0, 1, 2, 3, 7]","[0, 1, 2, 3, 4, 7, 8, 9]","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]",[1],"[0, 1, 2, 3, 4, 7, 9, 10]","[0, 1, 2, 6, 7]","[0, 1, 2, 3, 4, 6, 7, 8, 9]","[0, 1, 2, 4, 5]","[0, 1]","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]","[0, 7]","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]","[3, 4]","[0, 1, 2, 4, 7, 9]","[0, 2, 7]","[0, 1, 2, 3, 4, 7, 9]","[0, 1, 2, 3, 4, 7, 9]",[0],"[0, 1, 2, 3, 7]","[0, 1, 2, 3, 4, 7, 9]","[0, 2, 3, 7]","[0, 1, 2, 3, 4, 7]","[0, 1, 2, 3, 4, 7]","[0, 2, 3, 7]","[0, 1, 2, 4, 7, 9]","[0, 1, 2, 3, 7]","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]","[0, 1, 7]","[0, 1, 2, 7]","[0, 1, 7]","[0, 1, 3]","[0, 1, 2, 4, 7, 9]",[0],"[0, 1, 2, 3, 4, 7, 9, 10]",[0],[0],"[0, 1, 7, 9]","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]",[2],"[0, 1, 2, 3, 4, 7]","[0, 2]","[0, 1, 2, 4]","[0, 1, 2, 4, 7]","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]",[1],"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]","[0, 1, 3]","[0, 1, 2, 3, 4, 5, 6, 7, 9]","[0, 1, 2, 3, 4, 7, 9]","[0, 1, 2, 3, 4, 5, 7, 9]","[0, 1, 2, 3, 4, 7, 9]","[0, 1, 7]","[0, 1, 2, 3, 4, 5, 7, 9, 10]","[0, 1, 2, 3, 4, 5, 7, 8, 9, 10]","[0, 1, 2, 3, 4, 7, 9, 10]","[0, 1, 2, 3, 7]",[4],"[0, 1, 2, 4, 7, 9]","[0, 1, 2, 7, 9]","[0, 1, 2, 3, 7, 9]",[0],"[0, 2, 7]","[0, 1, 2, 3, 4, 7, 9]","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]","[0, 1, 2, 4]","[0, 1, 7, 9]","[0, 1, 2, 7]","[0, 2, 3, 7]","[0, 2, 3, 4, 7, 9]","[0, 1, 2, 3, 4, 7, 9]","[0, 1, 2, 3, 6, 7]","[0, 1, 2, 3, 4, 7]","[0, 1, 2, 3, 4, 7]","[0, 1, 4, 7]","[0, 1, 7]","[0, 9]","[0, 7]","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]","[0, 1, 2, 4, 6, 7, 9]","[0, 1, 2, 3, 4, 7, 9]","[0, 1, 2, 3, 4, 5, 6, 7, 9]","[0, 1, 2, 5, 7]","[1, 2, 7, 9]","[0, 1, 2, 3, 6, 7, 9]","[0, 3, 4, 5, 9]","[0, 1, 2, 3, 7, 8]","[0, 1, 2, 3, 6]","[1, 9]",[4],"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]","[0, 2, 7]","[0, 1]","[0, 1, 3]",[1],[0],"[0, 1, 2, 3, 4, 7, 9]","[0, 7]","[0, 1, 2, 3, 4, 5, 7, 9]"


{'mean': 4.869047619047619,
 'median': 5.0,
 'min': 1,
 'max': 11,
 'unique_vowels': 11,
 'root_combine_0': 155,
 'unique_roots': 168,
 'root_combine_not_0': '[26, 28, 33, 34, 73, 82, 108, 114, 126, 152, 157, 158, 163]',
 'root_combine_all': [13, 23, 64, 72, 79, 81, 96, 107, 113, 115, 133, 147]}

grapheme_root,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167
consonant_diacritic,[0],[0],"[0, 4]","[0, 1]","[0, 1]",[0],"[0, 1]",[0],[0],"[0, 1, 4]",[0],[0],[0],"[0, 1, 2, 4, 5, 6]",[0],"[0, 5]",[0],[0],"[0, 4]",[0],[0],"[0, 2]","[0, 1, 2, 4, 5]","[0, 1, 2, 4, 5, 6]",[0],"[0, 4]",[0],[0],"[0, 4]","[0, 1, 2, 3, 4, 5]",[0],[0],[0],[0],[0],[0],[0],[0],"[0, 1, 2, 4]",[0],[0],[0],"[0, 1, 2, 4]","[0, 2, 3, 4, 5]",[0],[0],[0],[0],"[0, 1, 2]",[0],[0],[0],[0],"[0, 2, 4, 5, 6]",[0],"[0, 1, 4]","[0, 2, 4, 5]",[0],"[0, 4]","[0, 2, 4]",[0],[0],[0],[0],"[0, 1, 2, 4, 5, 6]","[0, 2, 4]",[0],[0],[0],[0],"[0, 4]","[0, 2, 3, 4, 5]","[0, 1, 2, 4, 5, 6]",[0],"[0, 2]","[0, 2]","[0, 2, 4]","[0, 5]",[0],"[0, 1, 2, 4, 5]","[0, 2]","[0, 2, 4]",[0],"[0, 5]",[0],"[0, 5]","[0, 2, 4, 5]",[0],[0],"[0, 4, 5]",[0],"[0, 4, 5]","[0, 4]",[0],[0],[0],"[0, 1, 2, 4, 5, 6]",[0],[0],[0],[0],"[0, 4]",[0],"[0, 1, 2, 4, 5, 6]",[0],[0],"[0, 4]","[0, 1, 2, 3, 4, 5, 6]",[0],[0],[0],"[0, 2]","[0, 4]","[0, 1, 2, 4, 5]",[0],"[0, 2, 4, 5]",[0],"[0, 5]",[0],"[0, 5]","[0, 2]",[0],"[0, 2, 4]","[0, 1, 4]","[0, 2, 4]",[0],[0],[0],"[0, 2]",[0],[0],[0],[0],"[0, 2, 4, 5]",[0],[0],"[0, 2]",[0],[0],"[0, 2, 4]","[0, 5]","[0, 4, 5]","[0, 4]",[0],"[0, 5]",[0],[0],"[0, 1, 2, 4, 5]","[0, 4, 5]","[0, 2, 4, 5, 6]","[0, 5]","[0, 2, 4]",[0],"[0, 4, 5]",[0],[0],[0],[0],[0],"[0, 1, 2, 4, 5]",[0],[0],[0],[0],[0],[0],[0],"[0, 4]"


{'mean': 1.9583333333333333,
 'median': 1.0,
 'min': 1,
 'max': 7,
 'unique_consonants': 7,
 'root_combine_0': 168,
 'unique_roots': 168,
 'root_combine_not_0': '[]',
 'root_combine_all': [107]}

### Combination Matrices

This is the full list of which Grapheme Roots combine with which Vowels and Consonant Diacritics

In [9]:
combination_matrix(dataset, x='consonant_diacritic', y='vowel_diacritic', z='grapheme_root', format=', ')

consonant_diacritic,0.0,1.0,2.0,3.0,4.0,5.0,6.0
vowel_diacritic,,,,,,,
0,"0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0, 20.0, 21.0, 22.0, 23.0, 24.0, 25.0, 27.0, 29.0, 30.0, 31.0, 32.0, 35.0, 36.0, 37.0, 38.0, 39.0, 40.0, 41.0, 42.0, 43.0, 44.0, 45.0, 46.0, 47.0, 48.0, 49.0, 50.0, 51.0, 52.0, 53.0, 54.0, 55.0, 56.0, 57.0, 58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65.0, 66.0, 67.0, 68.0, 69.0, 70.0, 71.0, 72.0, 74.0, 75.0, 76.0, 77.0, 78.0, 79.0, 80.0, 81.0, 83.0, 84.0, 85.0, 86.0, 87.0, 88.0, 89.0, 90.0, 91.0, 92.0, 93.0, 94.0, 95.0, 96.0, 97.0, 98.0, 99.0, 100.0, 101.0, 102.0, 103.0, 104.0, 105.0, 106.0, 107.0, 109.0, 110.0, 111.0, 112.0, 113.0, 115.0, 116.0, 117.0, 118.0, 119.0, 120.0, 121.0, 122.0, 123.0, 124.0, 125.0, 127.0, 128.0, 129.0, 130.0, 131.0, 132.0, 133.0, 134.0, 135.0, 136.0, 137.0, 138.0, 139.0, 140.0, 141.0, 142.0, 143.0, 144.0, 145.0, 146.0, 147.0, 148.0, 149.0, 150.0, 151.0, 153.0, 154.0, 155.0, 156.0, 159.0, 160.0, 161.0, 162.0, 164.0, 165.0, 166.0, 167.0","3.0, 4.0, 6.0, 9.0, 96.0","13.0, 21.0, 22.0, 23.0, 29.0, 38.0, 42.0, 43.0, 48.0, 53.0, 56.0, 59.0, 64.0, 65.0, 71.0, 72.0, 74.0, 75.0, 76.0, 79.0, 80.0, 81.0, 86.0, 96.0, 103.0, 107.0, 111.0, 113.0, 115.0, 120.0, 122.0, 124.0, 128.0, 133.0, 136.0, 139.0, 147.0, 149.0, 159.0","29.0, 43.0, 71.0, 107.0","13.0, 18.0, 22.0, 23.0, 38.0, 43.0, 53.0, 55.0, 58.0, 59.0, 64.0, 65.0, 70.0, 71.0, 72.0, 76.0, 79.0, 81.0, 86.0, 89.0, 91.0, 96.0, 107.0, 113.0, 115.0, 122.0, 124.0, 133.0, 139.0, 141.0, 142.0, 147.0, 151.0, 159.0","13.0, 15.0, 23.0, 29.0, 43.0, 53.0, 56.0, 64.0, 72.0, 85.0, 86.0, 89.0, 91.0, 96.0, 103.0, 107.0, 113.0, 115.0, 117.0, 119.0, 133.0, 141.0, 147.0, 148.0, 149.0, 150.0, 159.0","64.0, 72.0"
1,"13.0, 14.0, 15.0, 16.0, 17.0, 18.0, 21.0, 22.0, 23.0, 24.0, 25.0, 28.0, 29.0, 31.0, 32.0, 34.0, 35.0, 36.0, 38.0, 39.0, 40.0, 41.0, 42.0, 43.0, 44.0, 46.0, 47.0, 48.0, 49.0, 50.0, 52.0, 53.0, 54.0, 55.0, 56.0, 57.0, 58.0, 59.0, 60.0, 61.0, 62.0, 64.0, 65.0, 66.0, 67.0, 69.0, 70.0, 71.0, 72.0, 73.0, 74.0, 75.0, 76.0, 77.0, 78.0, 79.0, 81.0, 83.0, 85.0, 86.0, 88.0, 89.0, 91.0, 92.0, 94.0, 95.0, 96.0, 97.0, 98.0, 99.0, 100.0, 101.0, 103.0, 106.0, 107.0, 109.0, 111.0, 112.0, 113.0, 114.0, 115.0, 116.0, 117.0, 118.0, 119.0, 120.0, 121.0, 122.0, 123.0, 124.0, 125.0, 127.0, 128.0, 129.0, 132.0, 133.0, 134.0, 135.0, 136.0, 139.0, 140.0, 141.0, 142.0, 143.0, 144.0, 147.0, 148.0, 149.0, 150.0, 151.0, 152.0, 153.0, 155.0, 156.0, 157.0, 159.0, 161.0, 162.0, 163.0, 165.0, 167.0","13.0, 22.0, 23.0, 29.0, 38.0, 42.0, 48.0, 55.0, 64.0, 72.0, 79.0, 96.0, 103.0, 107.0, 113.0, 123.0, 147.0, 159.0","13.0, 22.0, 23.0, 29.0, 38.0, 42.0, 43.0, 53.0, 56.0, 59.0, 64.0, 71.0, 72.0, 74.0, 79.0, 81.0, 107.0, 113.0, 115.0, 122.0, 124.0, 133.0, 139.0, 147.0, 151.0",,"2.0, 9.0, 13.0, 18.0, 22.0, 23.0, 25.0, 28.0, 29.0, 38.0, 42.0, 43.0, 53.0, 55.0, 56.0, 59.0, 64.0, 71.0, 72.0, 79.0, 81.0, 91.0, 92.0, 96.0, 101.0, 103.0, 106.0, 107.0, 112.0, 113.0, 115.0, 122.0, 123.0, 124.0, 133.0, 139.0, 141.0, 147.0, 148.0, 149.0, 153.0, 159.0, 167.0","13.0, 23.0, 29.0, 53.0, 56.0, 64.0, 72.0, 77.0, 83.0, 86.0, 89.0, 96.0, 103.0, 107.0, 113.0, 115.0, 119.0, 133.0, 144.0, 147.0, 149.0, 150.0, 153.0, 159.0","13.0, 23.0, 53.0, 96.0, 103.0, 107.0, 149.0"
2,"13.0, 14.0, 15.0, 16.0, 17.0, 18.0, 21.0, 22.0, 23.0, 25.0, 27.0, 28.0, 29.0, 30.0, 31.0, 32.0, 33.0, 34.0, 35.0, 36.0, 37.0, 38.0, 39.0, 40.0, 42.0, 43.0, 44.0, 47.0, 48.0, 50.0, 51.0, 52.0, 53.0, 54.0, 55.0, 56.0, 57.0, 58.0, 59.0, 61.0, 62.0, 64.0, 65.0, 66.0, 68.0, 69.0, 70.0, 71.0, 72.0, 74.0, 75.0, 76.0, 77.0, 79.0, 81.0, 83.0, 84.0, 85.0, 86.0, 88.0, 89.0, 90.0, 91.0, 92.0, 93.0, 94.0, 95.0, 96.0, 98.0, 101.0, 103.0, 107.0, 108.0, 109.0, 110.0, 111.0, 112.0, 113.0, 115.0, 117.0, 118.0, 119.0, 120.0, 122.0, 123.0, 124.0, 125.0, 127.0, 128.0, 129.0, 131.0, 132.0, 133.0, 134.0, 136.0, 137.0, 138.0, 139.0, 141.0, 142.0, 147.0, 148.0, 149.0, 150.0, 151.0, 152.0, 153.0, 155.0, 156.0, 159.

### Visualizing Bengali

In [10]:
combination_matrix(dataset, x='consonant_diacritic', y='vowel_diacritic', z='grapheme', format=' ')

consonant_diacritic,0.0,1.0,2.0,3.0,4.0,5.0,6.0
vowel_diacritic,,,,,,,
0,ং ঃ অ আ ই ঈ উ ঊ ঋ এ ঐ ও ঔ ক ক্ক ক্ট ক্ত ক্ল ক্ষ ক্ষ্ণ ক্ষ্ম ক্স খ গ গ্ধ গ্ন গ্ম ঘ ঘ্ন ঙ ঙ্ক ঙ্খ ঙ্গ ঙ্ঘ চ চ্চ চ্ছ চ্ছ্ব ছ জ জ্জ জ্জ্ব জ্ঞ জ্ব ঝ ঞ ঞ্চ ঞ্ছ ঞ্জ ট ট্ট ঠ ড ড্ড ঢ ণ ণ্ট ণ্ঠ ণ্ড ণ্ণ ত ত্ত ত্ত্ব ত্থ ত্ন ত্ব ত্ম থ দ দ্দ দ্ধ দ্ব দ্ভ দ্ম ধ ধ্ব ন ন্ট ন্ঠ ন্ড ন্ত ন্ত্ব ন্থ ন্দ ন্দ্ব ন্ধ ন্ন ন্ব ন্ম ন্স প প্ট প্ত প্ন প্প প্ল প্স ফ ফ্ট ফ্ফ ফ্ল ব ব্দ ব্ধ ব্ব ব্ল ভ ম ম্ন ম্প ম্ব ম্ভ ম্ম ম্ল য র ল ল্ক ল্ট ল্ড ল্প ল্ব ল্ম ল্ল শ শ্চ শ্ন শ্ব শ্ম শ্ল ষ ষ্ক ষ্ট ষ্ঠ ষ্ণ ষ্প ষ্ফ ষ্ম স স্ক স্ট স্ত স্থ স্প স্ফ স্ব স্ম হ হ্ন হ্ব হ্ম ৎ ড় ঢ় য়,আঁ ইঁ উঁ এঁ পঁ,র্ক র্ক্স র্খ র্গ র্ঘ র্চ র্ছ র্জ র্ঝ র্ট র্ড র্ণ র্ত র্ত্ত র্থ র্দ র্দ্দ র্দ্ধ র্দ্ব র্দ্র র্ধ র্ধ্ব র্ন র্ন্ত র্প র্ফ র্ব র্ব্ব র্ভ র্ম র্ম্ম র্য র্ল র্ল্ড র্শ র্শ্ব র্ষ র্স র্স্ট র্হ,র্ঘ্য র্জ্য র্থ্য র্ব্য,ক্য ক্ষ্য খ্য গ্য চ্য জ্য ট্য ঠ্য ঢ্য ণ্য ত্ত্য ত্ম্য ত্য থ্য দ্ব্য দ্য ধ্য ন্ত্য ন্দ্য ন্ধ্য ন্য প্য ব্য ভ্য ম্য য্য ল্য শ্য ষ্ট্য ষ্ঠ্য ষ্য স্থ্য স্য হ্য,ক্ট্র ক্র গ্র ঘ্র জ্র ট্র ড্র ত্র দ্র ন্ড্র ন্ত্র ন্দ্র ন্ধ্র প্র ফ্র ব্র ভ্র ম্প্র ম্ভ্র ম্র শ্র ষ্ট্র স্ক্র স্ট্র স্ত্র স্র হ্র,ত্র্য দ্র্য
1,কা ক্কা ক্টা ক্তা ক্লা ক্ষা ক্সা খা গা গ্ধা গ্না গ্লা ঘা ঙা ঙ্কা ঙ্ক্ষা ঙ্খা ঙ্গা চা চ্চা চ্ছা চ্ছ্বা ছা জা জ্জা জ্ঞা জ্বা ঝা ঞা ঞ্চা ঞ্জা টা ট্টা ঠা ডা ড্ডা ঢা ণা ণ্টা ণ্ঠা ণ্ডা তা ত্তা ত্ত্বা ত্থা ত্বা ত্মা থা দা দ্ঘা দ্দা দ্ধা দ্বা দ্ভা দ্মা ধা না ন্টা ন্ডা ন্তা ন্থা ন্দা ন্ধা ন্না ন্মা ন্সা পা প্টা প্তা প্না প্পা প্লা ফা ফ্লা বা ব্দা ব্বা ব্লা ভা ভ্লা মা ম্না ম্পা ম্বা ম্ভা ম্মা ম্লা যা রা লা ল্কা ল্টা ল্ডা ল্পা ল্লা শা শ্চা শ্না শ্বা ষা ষ্কা ষ্টা ষ্ঠা ষ্ণা ষ্পা সা স্কা স্টা স্তা স্থা স্না স্পা স্বা স্মা স্লা হা হ্বা হ্মা হ্লা ড়া য়া,কাঁ খাঁ গাঁ ঘাঁ চাঁ ছাঁ ঝাঁ ঠাঁ তাঁ দাঁ ধাঁ পাঁ ফাঁ বাঁ ভাঁ রাঁ সাঁ হাঁ,র্কা র্খা র্গা র্ঘা র্চা র্ছা র্জা র্টা র্ডা র্ণা র্তা র্থা র্দা র্দ্দা র্ধা র্না র্বা র্ভা র্মা র্যা র্লা র্শা র্ষা র্সা র্স্থা,,অ্যা এ্যা ক্যা ক্ষ্যা খ্যা গ্ন্যা গ্যা গ্ল্যা ঘ্যা চ্যা ছ্যা জ্যা ট্যা ঠ্যা ড্যা ণ্যা ত্যা থ্যা দ্যা ধ্যা ন্ধ্যা ন্ন্যা ন্যা প্যা প্ল্যা ফ্যা ফ্ল্যা ব্যা ব্ল্যা ভ্যা ম্যা য্যা র‍্যা ল্যা শ্যা ষ্ট্যা ষ্যা স্ক্যা স্ট্যা স্প্যা স্যা হ্যা য়্যা,ক্রা গ্রা ঘ্রা ট্রা ড্রা ত্রা দ্ভ্রা দ্রা ন্ট্রা ন্ত্রা ন্দ্রা প্রা ফ্রা ব্রা ভ্রা ম্ভ্রা ম্রা শ্রা ষ্প্রা স্ট্রা স্ত্রা স্প্রা স্রা হ্রা,ক্র্যা গ্র্যা ট্র্যা প্র্যা ফ্র্যা ব্র্যা স্ট্র্যা
2,কি ক্কি ক্টি ক্তি ক্লি ক্ষি ক্সি খি গি গ্নি গ্মি গ্লি ঘি ঘ্নি ঙি ঙ্কি ঙ্ক্তি ঙ্ক্ষি ঙ্খি ঙ্গি ঙ্ঘি চি চ্চি চ্ছি ছি জি জ্জি জ্বি ঝি ঞ্চি ঞ্ছি ঞ্জি টি ট্টি ঠি ডি ড্ডি ঢি ণি ণ্ঠি ণ্ডি তি ত্তি ত্ত্বি ত্নি ত্বি ত্মি থি দি দ্দি দ্ধি দ্বি দ্ভি ধি নি ন্টি ন্ঠি ন্ডি ন্তি ন্থি ন্দি ন্দ্বি ন্ধি ন্নি ন্বি ন্মি ন্সি পি প্তি প্লি ফি বি ব্জি ব্দি ব্ধি ব্বি ব্লি ভি মি ম্পি ম্বি ম্ভি ম্মি যি রি লি ল্কি ল্টি ল্ডি ল্পি ল্মি ল্লি শি শ্চি শ্বি শ্মি শ্লি ষি ষ্টি ষ্ঠি সি স্কি স্টি স্তি স্থি স্নি স্পি স্বি স্মি হি হ্নি ড়ি য়ি,ছিঁ পিঁ সিঁ,র্কি র্গি র্চি র্জি র্টি র্ডি র্ণি র্তি র্থি র্দি র্দ্বি র্ধি র্নি র্পি র্ফি র্বি র্ভি র্মি র্লি র্শি র্শ্বি র্ষি র্সি র্হি,,গ্যি জ্যি ত্যি দ্যি ধ্যি ব্যি শ্যি হ্যি,ক্রি খ্রি গ্রি ট্রি ড্রি ত্রি থ্রি দ্রি ন্ট্রি ন্ত্রি ন্দ্রি প্রি ফ্রি ব্রি শ্রি ষ্ক্রি স্ক্রি স্ট্রি স্ত্রি স্প্রি,
3,কী ক্ষী ক্ষ্মী খী গী গ্নী ঙ্কী ঙ্গী চী জী জ্জী জ্ঞী ঞ্জী টী ঠী ডী ড্ডী ণী ণ্ডী তী ত্তী ত্ত্বী ত্নী ত্বী ত্মী থী দী দ্দী দ্বী ধী নী ন্জী ন্ডী ন্তী ন্থী ন্দী ন্দ্বী ন্ধী ন্নী ন্বী ন্সী পী প্পী ফী বী ব্দী ভী মী ম্নী ম্বী ম্ভী ম্মী যী রী লী ল্কী ল্পী ল্লী শী শ্মী শ্লী ষী ষ্টী ষ্ঠী সী স্টী স্তী স্পী স্ফী স্বী স্মী হী হ্মী ড়ী য়ী,,র্কী র্ক্সী র্গী র্জী র্ণী র্তী র্ত্তী র্ত্রী র্থী র্দী র্নী র্মী র্শী র্ষী র্সী,,,ক্রী খ্রী গ্রী ত্রী দ্রী ন্ত্রী ন্দ্রী প্রী ফ্রী ব্রী ম্প্রী শ্রী ষ্ক্রী ষ্ট্রী স্ট্রী স্ত্রী,
4,কু ক্ষু খু গু গ্লু ঘু ঙু ঙ্কু ঙ্গু চু চ্চু চ্ছু ছু জু জ্জু ঝু ঞ্চু ঞ্জু টু ঠু ডু ঢু ণু ণ্টু ণ্ডু তু থু দু দ্দু দ্বু দ্ভু ধু নু ন্জু ন্টু ন্ডু ন্তু ন্দু ন্ধু ন্নু ন্মু পু প্লু ফু বু ব্দু ব্বু ব্লু ভু মু ম্পু ম্বু ম্ভু ম্মু যু রু লু ল্গু ল্টু ল্লু শু শ্চু শ্লু ষু ষ্টু ষ্ঠু ষ্ণু সু স্কু স্টু স্তু স্ফু স্সু হু ড়ু য়ু,কুঁ খুঁ গুঁ ছুঁ ঝুঁ পুঁ ফুঁ হুঁ,র্কু র্গু র্ঘু র্জু র্টু র্তু র্দু 

In [11]:
combination_matrix(dataset, x='grapheme_root', y='vowel_diacritic', z='grapheme', format=' ')

grapheme_root,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0,12.0,13.0,14.0,15.0,16.0,17.0,18.0,19.0,20.0,21.0,22.0,23.0,24.0,25.0,26.0,27.0,28.0,29.0,30.0,31.0,32.0,33.0,34.0,35.0,36.0,37.0,38.0,39.0,40.0,41.0,42.0,43.0,44.0,45.0,46.0,47.0,48.0,49.0,50.0,51.0,52.0,53.0,54.0,55.0,56.0,57.0,58.0,59.0,60.0,61.0,62.0,63.0,64.0,65.0,66.0,67.0,68.0,69.0,70.0,71.0,72.0,73.0,74.0,75.0,76.0,77.0,78.0,79.0,80.0,81.0,82.0,83.0,84.0,85.0,86.0,87.0,88.0,89.0,90.0,91.0,92.0,93.0,94.0,95.0,96.0,97.0,98.0,99.0,100.0,101.0,102.0,103.0,104.0,105.0,106.0,107.0,108.0,109.0,110.0,111.0,112.0,113.0,114.0,115.0,116.0,117.0,118.0,119.0,120.0,121.0,122.0,123.0,124.0,125.0,126.0,127.0,128.0,129.0,130.0,131.0,132.0,133.0,134.0,135.0,136.0,137.0,138.0,139.0,140.0,141.0,142.0,143.0,144.0,145.0,146.0,147.0,148.0,149.0,150.0,151.0,152.0,153.0,154.0,155.0,156.0,157.0,158.0,159.0,160.0,161.0,162.0,163.0,164.0,165.0,166.0,167.0
vowel_diacritic,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,ং,ঃ,অ,আ আঁ,ই ইঁ,ঈ,উ উঁ,ঊ,ঋ,এ এঁ,ঐ,ও,ঔ,ক ক্য ক্র র্ক,ক্ক,ক্ট ক্ট্র,ক্ত,ক্ল,ক্ষ ক্ষ্য,ক্ষ্ণ,ক্ষ্ম,ক্স র্ক্স,খ খ্য র্খ,গ গ্য গ্র র্গ,গ্ধ,গ্ন,,গ্ম,,ঘ ঘ্র র্ঘ র্ঘ্য,ঘ্ন,ঙ,ঙ্ক,,,ঙ্খ,ঙ্গ,ঙ্ঘ,চ চ্য র্চ,চ্চ,চ্ছ,চ্ছ্ব,ছ র্ছ,জ জ্য জ্র র্জ র্জ্য,জ্জ,জ্জ্ব,জ্ঞ,জ্ব,ঝ র্ঝ,ঞ,ঞ্চ,ঞ্ছ,ঞ্জ,ট ট্য ট্র র্ট,ট্ট,ঠ ঠ্য,ড ড্র র্ড,ড্ড,ঢ ঢ্য,ণ ণ্য র্ণ,ণ্ট,ণ্ঠ,ণ্ড,ণ্ণ,ত ত্য ত্র ত্র্য র্ত,ত্ত ত্ত্য র্ত্ত,ত্ত্ব,ত্থ,ত্ন,ত্ব,ত্ম ত্ম্য,থ থ্য র্থ র্থ্য,দ দ্য দ্র দ্র্য র্দ র্দ্র,,দ্দ র্দ্দ,দ্ধ র্দ্ধ,দ্ব দ্ব্য র্দ্ব,দ্ভ,দ্ম,ধ ধ্য র্ধ,ধ্ব র্ধ্ব,ন ন্য র্ন,,ন্ট,ন্ঠ,ন্ড ন্ড্র,ন্ত ন্ত্য ন্ত্র র্ন্ত,ন্ত্ব,ন্থ,ন্দ ন্দ্য ন্দ্র,ন্দ্ব,ন্ধ ন্ধ্য ন্ধ্র,ন্ন,ন্ব,ন্ম,ন্স,প পঁ প্য প্র র্প,প্ট,প্ত,প্ন,প্প,প্ল,প্স,ফ ফ্র র্ফ,ফ্ট,ফ্ফ,ফ্ল,ব ব্য ব্র র্ব র্ব্য,,ব্দ,ব্ধ,ব্ব র্ব্ব,ব্ল,ভ ভ্য ভ্র র্ভ,,ম ম্য ম্র র্ম,ম্ন,ম্প ম্প্র,ম্ব,ম্ভ ম্ভ্র,ম্ম র্ম্ম,ম্ল,য য্য র্য,র,র্ল ল ল্য,ল্ক,,ল্ট,র্ল্ড ল্ড,ল্প,ল্ব,ল্ম,ল্ল,র্শ শ শ্য শ্র,শ্চ,শ্ন,র্শ্ব শ্ব,শ্ম,শ্ল,র্ষ ষ ষ্য,ষ্ক,ষ্ট ষ্ট্য ষ্ট্র,ষ্ঠ ষ্ঠ্য,ষ্ণ,ষ্প,ষ্ফ,ষ্ম,র্স স স্য স্র,স্ক স্ক্র,র্স্ট স্ট স্ট্র,স্ত স্ত্র,স্থ স্থ্য,,স্প,স্ফ,স্ব,স্ম,,,র্হ হ হ্য হ্র,হ্ন,হ্ব,হ্ম,,ৎ,ড়,ঢ়,য়
1,,,অ্যা,,,,,,,এ্যা,,,,কা কাঁ ক্যা ক্রা ক্র্যা র্কা,ক্কা,ক্টা,ক্তা,ক্লা,ক্ষা ক্ষ্যা,,,ক্সা,খা খাঁ খ্যা র্খা,গা গাঁ গ্যা গ্রা গ্র্যা র্গা,গ্ধা,গ্না গ্ন্যা,,,গ্লা গ্ল্যা,ঘা ঘাঁ ঘ্যা ঘ্রা র্ঘা,,ঙা,ঙ্কা,,ঙ্ক্ষা,ঙ্খা,ঙ্গা,,চা চাঁ চ্যা র্চা,চ্চা,চ্ছা,চ্ছ্বা,ছা ছাঁ ছ্যা র্ছা,জা জ্যা র্জা,জ্জা,,জ্ঞা,জ্বা,ঝা ঝাঁ,ঞা,ঞ্চা,,ঞ্জা,টা ট্যা ট্রা ট্র্যা র্টা,ট্টা,ঠা ঠাঁ ঠ্যা,ডা ড্যা ড্রা র্ডা,ড্ডা,ঢা,ণা ণ্যা র্ণা,ণ্টা,ণ্ঠা,ণ্ডা,,তা তাঁ ত্যা ত্রা র্তা,ত্তা,ত্ত্বা,ত্থা,,ত্বা,ত্মা,থা থ্যা র্থা,দা দাঁ দ্যা দ্রা র্দা,দ্ঘা,দ্দা র্দ্দা,দ্ধা,দ্বা,দ্ভা দ্ভ্রা,দ্মা,ধা ধাঁ ধ্যা র্ধা,,না ন্যা র্না,,ন্টা ন্ট্রা,,ন্ডা,ন্তা ন্ত্রা,,ন্থা,ন্দা ন্দ্রা,,ন্ধা ন্ধ্যা,ন্না ন্ন্যা,,ন্মা,ন্সা,পা পাঁ প্যা প্রা প্র্যা,প্টা,প্তা,প্না,প্পা,প্লা প্ল্যা,,ফা ফাঁ ফ্যা ফ্রা ফ্র্যা,,,ফ্লা ফ্ল্যা,বা বাঁ ব্যা ব্রা ব্র্যা র্বা,,ব্দা,,ব্বা,ব্লা ব্ল্যা,ভা ভাঁ ভ্যা ভ্রা র্ভা,ভ্লা,মা ম্যা ম্রা র্মা,ম্না,ম্পা,ম্বা,ম্ভা ম্ভ্রা,ম্মা,ম্লা,যা য্যা র্যা,রা রাঁ র‍্যা,র্লা লা ল্যা,ল্কা,,ল্টা,ল্ডা,ল্পা,,,ল্লা,র্শা শা শ্যা শ্রা,শ্চা,শ্না,শ্বা,,,র্ষা ষা ষ্যা,ষ্কা,ষ্টা ষ্ট্যা,ষ্ঠা,ষ্ণা,ষ্পা ষ্প্রা,,,র্সা সা সাঁ স্যা স্রা,স্কা স্ক্যা,স্টা স্ট্যা স্ট্রা স্ট্র্যা,স্তা স্ত্রা,র্স্থা স্থা,স্না,স্পা স্প্যা স্প্রা,,স্বা,স্মা,স্লা,,হা হাঁ হ্যা হ্রা,,হ্বা,হ্মা,হ্লা,,ড়া,,য়া য়্যা
2,,,,,,,,,,,,,,কি ক্রি র্কি,ক্কি,ক্টি,ক্তি,ক্লি,ক্ষি,,,ক্সি,খি খ্রি,গি গ্যি গ্রি র্গি,,গ্নি,,গ্মি,গ্লি,ঘি,ঘ্নি,ঙি,ঙ্কি,ঙ্ক্তি,ঙ্ক্ষি,ঙ্খি,ঙ্গি,ঙ্ঘি,চি র্চি,চ্চি,চ্ছি,,ছি ছিঁ,জি জ্যি র্জি,জ্জি,,,জ্বি,ঝি,,ঞ্চি,ঞ্ছি,ঞ্জি,টি ট্রি র্টি,ট্টি,ঠি,ডি ড্রি র্ডি,ড্ডি,ঢি,ণি র্ণি,,ণ্ঠি,ণ্ডি,,তি ত্যি ত্রি র্তি,ত্তি,ত্ত্বি,,ত্নি,ত্বি,ত্মি,থি থ্রি র্থি,দি দ্যি দ্রি র্দি,,দ্দি,দ্ধি,দ্বি র্দ্বি,দ্ভি,,ধি ধ্যি র্ধি,,নি র্নি,,ন্টি ন্ট্রি,ন্ঠি,ন্ডি,ন্তি ন্ত্রি,,ন্থি,ন্দি ন্দ্রি,ন্দ্বি,ন্ধি,ন্নি,ন্বি,ন্মি,ন্সি,পি পিঁ প্রি র্পি,,প্তি,,,প্লি,,ফি ফ্রি র্ফি,,,,বি ব্যি ব্রি র্বি,ব্জি,ব্দি,ব্ধি,ব্বি,ব্লি,ভি র্ভি,,মি র্মি,,ম্পি,ম্বি

In [12]:
combination_matrix(dataset, x='grapheme_root', y='consonant_diacritic', z='grapheme', format=' ')

grapheme_root,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0,12.0,13.0,14.0,15.0,16.0,17.0,18.0,19.0,20.0,21.0,22.0,23.0,24.0,25.0,26.0,27.0,28.0,29.0,30.0,31.0,32.0,33.0,34.0,35.0,36.0,37.0,38.0,39.0,40.0,41.0,42.0,43.0,44.0,45.0,46.0,47.0,48.0,49.0,50.0,51.0,52.0,53.0,54.0,55.0,56.0,57.0,58.0,59.0,60.0,61.0,62.0,63.0,64.0,65.0,66.0,67.0,68.0,69.0,70.0,71.0,72.0,73.0,74.0,75.0,76.0,77.0,78.0,79.0,80.0,81.0,82.0,83.0,84.0,85.0,86.0,87.0,88.0,89.0,90.0,91.0,92.0,93.0,94.0,95.0,96.0,97.0,98.0,99.0,100.0,101.0,102.0,103.0,104.0,105.0,106.0,107.0,108.0,109.0,110.0,111.0,112.0,113.0,114.0,115.0,116.0,117.0,118.0,119.0,120.0,121.0,122.0,123.0,124.0,125.0,126.0,127.0,128.0,129.0,130.0,131.0,132.0,133.0,134.0,135.0,136.0,137.0,138.0,139.0,140.0,141.0,142.0,143.0,144.0,145.0,146.0,147.0,148.0,149.0,150.0,151.0,152.0,153.0,154.0,155.0,156.0,157.0,158.0,159.0,160.0,161.0,162.0,163.0,164.0,165.0,166.0,167.0
consonant_diacritic,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,ং,ঃ,অ,আ,ই,ঈ,উ,ঊ,ঋ,এ,ঐ,ও,ঔ,ক কা কি কী কু কূ কৃ কে কৈ কো কৌ,ক্ক ক্কা ক্কি ক্কে ক্কো,ক্ট ক্টা ক্টি ক্টে ক্টো,ক্ত ক্তা ক্তি ক্তৃ ক্তে ক্তো,ক্ল ক্লা ক্লি ক্লে ক্লো,ক্ষ ক্ষা ক্ষি ক্ষী ক্ষু ক্ষে ক্ষো ক্ষৌ,ক্ষ্ণ ক্ষ্ণৌ,ক্ষ্ম ক্ষ্মী,ক্স ক্সা ক্সি ক্সে,খ খা খি খী খু খৃ খে খৈ খো,গ গা গি গী গু গূ গৃ গে গৈ গো গৌ,গ্ধ গ্ধা,গ্ন গ্না গ্নি গ্নী গ্নে,গ্বে,গ্ম গ্মি,গ্লা গ্লি গ্লু গ্লো,ঘ ঘা ঘি ঘু ঘূ ঘৃ ঘে ঘো,ঘ্ন ঘ্নি ঘ্নে,ঙ ঙা ঙি ঙু ঙে,ঙ্ক ঙ্কা ঙ্কি ঙ্কী ঙ্কু ঙ্কৃ ঙ্কে ঙ্কো,ঙ্ক্তি,ঙ্ক্ষা ঙ্ক্ষি,ঙ্খ ঙ্খা ঙ্খি,ঙ্গ ঙ্গা ঙ্গি ঙ্গী ঙ্গু ঙ্গে ঙ্গো,ঙ্ঘ ঙ্ঘি,চ চা চি চী চু চূ চে চৈ চো চৌ,চ্চ চ্চা চ্চি চ্চু,চ্ছ চ্ছা চ্ছি চ্ছু চ্ছে,চ্ছ্ব চ্ছ্বা,ছ ছা ছি ছু ছে ছো,জ জা জি জী জু জে জৈ জো,জ্জ জ্জা জ্জি জ্জী জ্জু জ্জে জ্জো,জ্জ্ব,জ্ঞ জ্ঞা জ্ঞী জ্ঞে,জ্ব জ্বা জ্বি,ঝ ঝা ঝি ঝু ঝে ঝো,ঞ ঞা,ঞ্চ ঞ্চা ঞ্চি ঞ্চু ঞ্চে,ঞ্ছ ঞ্ছি,ঞ্জ ঞ্জা ঞ্জি ঞ্জী ঞ্জু ঞ্জে,ট টা টি টী টু টূ টে টো,ট্ট ট্টা ট্টি ট্টো,ঠ ঠা ঠি ঠী ঠু ঠে ঠো,ড ডা ডি ডী ডু ডে ডো,ড্ড ড্ডা ড্ডি ড্ডী,ঢ ঢা ঢি ঢু ঢে ঢো ঢৌ,ণ ণা ণি ণী ণু ণে ণো,ণ্ট ণ্টা ণ্টু,ণ্ঠ ণ্ঠা ণ্ঠি ণ্ঠে,ণ্ড ণ্ডা ণ্ডি ণ্ডী ণ্ডু ণ্ডে,ণ্ণ,ত তা তি তী তু তূ তৃ তে তৈ তো তৌ,ত্ত ত্তা ত্তি ত্তী ত্তে ত্তো,ত্ত্ব ত্ত্বা ত্ত্বি ত্ত্বী ত্ত্বে,ত্থ ত্থা ত্থে,ত্ন ত্নি ত্নী ত্নে,ত্ব ত্বা ত্বি ত্বী ত্বে,ত্ম ত্মা ত্মি ত্মী,থ থা থি থী থু থে থৈ থো,দ দা দি দী দু দূ দৃ দে দৈ দো দৌ,দ্ঘা,দ্দ দ্দা দ্দি দ্দী দ্দু দ্দে দ্দো দ্দৌ,দ্ধ দ্ধা দ্ধি দ্ধৃ দ্ধে,দ্ব দ্বা দ্বি দ্বী দ্বু দ্বৃ দ্বে দ্বৈ দ্বো,দ্ভ দ্ভা দ্ভি দ্ভু দ্ভূ,দ্ম দ্মা,ধ ধা ধি ধী ধু ধূ ধৃ ধে ধৈ ধো ধৌ,ধ্ব,ন না নি নী নু নূ নৃ নে নৈ নো নৌ,ন্জী ন্জু,ন্ট ন্টা ন্টি ন্টু ন্টে ন্টো,ন্ঠ ন্ঠি ন্ঠে,ন্ড ন্ডা ন্ডি ন্ডী ন্ডু ন্ডে ন্ডো,ন্ত ন্তা ন্তি ন্তী ন্তু ন্তে ন্তো,ন্ত্ব,ন্থ ন্থা ন্থি ন্থী ন্থে,ন্দ ন্দা ন্দি ন্দী ন্দু ন্দে ন্দো,ন্দ্ব ন্দ্বি ন্দ্বী ন্দ্বে,ন্ধ ন্ধা ন্ধি ন্ধী ন্ধু ন্ধে,ন্ন ন্না ন্নি ন্নী ন্নু ন্নে,ন্ব ন্বি ন্বী ন্বে,ন্ম ন্মা ন্মি ন্মু ন্মে ন্মো,ন্স ন্সা ন্সি ন্সী ন্সে,প পা পি পী পু পূ পৃ পে পৈ পো পৌ,প্ট প্টা প্টে,প্ত প্তা প্তি প্তে,প্ন প্না প্নে,প্প প্পা প্পী,প্ল প্লা প্লি প্লু প্লে প্লো,প্স,ফ ফা ফি ফী ফু ফে ফো ফৌ,ফ্ট,ফ্ফ,ফ্ল ফ্লা ফ্লে ফ্লো,ব বা বি বী বু বৃ বে বৈ বো বৌ,ব্জি,ব্দ ব্দা ব্দি ব্দী ব্দু ব্দে,ব্ধ ব্ধি,ব্ব ব্বা ব্বি ব্বু,ব্ল ব্লা ব্লি ব্লু ব্লে,ভ ভা ভি ভী ভু ভূ ভৃ ভে ভৈ ভো ভৌ,ভ্লা,ম মা মি মী মু মূ মৃ মে মৈ মো মৌ,ম্ন ম্না ম্নী,ম্প ম্পা ম্পি ম্পু ম্পূ ম্পৃ ম্পে ম্পো,ম্ব ম্বা ম্বি ম্বী ম্বু ম্বে ম্বো,ম্ভ ম্ভা ম্ভি ম্ভী ম্ভু ম্ভূ ম্ভে ম্ভো,ম্ম ম্মা ম্মি ম্মী ম্মু ম্মে ম্মো,ম্ল ম্লা ম্লে,য যা যি যী যু যূ যে যো যৌ,র রা রি রী রু রূ রে রৈ রো রৌ,ল লা লি লী লু লে লো লৌ,ল্ক ল্কা ল্কি ল্কী ল্কে,ল্গু,ল্ট ল্টা ল্টি ল্টু ল্টে ল্টো,ল্ড ল্ডা ল্ডি ল্ডে ল্ডো,ল্প ল্পা ল্পি ল্পী ল্পে ল্পো,ল্ব,ল্ম ল্মি ল্মে,ল্ল ল্লা ল্লি ল্লী ল্লু ল্লে ল্লো,শ শা শি শী শু শূ শৃ শে শৈ শো শৌ,শ্চ শ্চা শ্চি শ্চু,শ্ন শ্না শ্নে শ্নো,শ্ব শ্বা শ্বি শ্বে,শ্ম শ্মি শ্মী শ্মে,শ্ল শ্লি শ্লী শ্লু শ্লে শ্লো,ষ ষা ষি ষী ষু ষে ষো,ষ্ক ষ্কা ষ্কৃ ষ্কে,ষ্ট ষ্টা ষ্টি ষ্টী ষ্টু ষ্টে,ষ্ঠ ষ্ঠা ষ্ঠি ষ্ঠী ষ্ঠু ষ্ঠে,ষ্ণ ষ্ণা ষ্ণু ষ্ণে,ষ্প ষ্পা ষ্পে,ষ্ফ ষ্ফো,ষ্ম ষ্মে,স সা সি সী সু সূ সৃ সে সৈ সো সৌ,

## Sanity Checking = Found Dataset BUG!

This combination_matrix lists 1292 unique grapheme combinations, which is 3 less than the 1295 unique graphemes listed in the training dataset. Something is WRONG!

Found a discrepency BUG is in the dataset. The following root/vowel/consonant keys have multiple unicode graphemes renderings! 

{'64-3-2': ['র্তী', 'র্ত্রী'],
 '64-7-2': ['র্তে', 'র্ত্রে'],
 '72-0-2': ['র্দ্র', 'র্দ']}

In [13]:
from itertools import chain
{
    "combinations": len(list(chain( 
        *combination_matrix(dataset, x='consonant_diacritic', y='vowel_diacritic', z='grapheme_root')
        .values.flatten() 
    ))),
    "unique_graphemes": unique['grapheme']
}

{'combinations': 1292, 'unique_graphemes': 1295}

- Confirm that there are no null or NaN values in the dataset

In [14]:
dataset.apply(lambda row: row.isnull()).sum()

image_id               0
grapheme_root          0
vowel_diacritic        0
consonant_diacritic    0
grapheme               0
dtype: int64

- Found the BUG! It is in the dataset!
- There is THREE sets of unique root/vowel/consonant keys that have multiple unicode renderings 

In [15]:
( 
    dataset
    .groupby(['grapheme_root', 'vowel_diacritic', 'consonant_diacritic'])
    .nunique(dropna=False) > 1 
).sum()

image_id               1292
grapheme_root             0
vowel_diacritic           0
consonant_diacritic       0
grapheme                  3
dtype: int64

In [16]:
( 
    dataset
    .groupby(['grapheme_root', 'vowel_diacritic', 'consonant_diacritic'])
    .nunique(dropna=False) > 1
).query("grapheme != False")

image_id  grapheme_root  \
grapheme_root vowel_diacritic consonant_diacritic                            
64            3               2                        True          False   
              7               2                        True          False   
72            0               2                        True          False   

                                                   vowel_diacritic  \
grapheme_root vowel_diacritic consonant_diacritic                    
64            3               2                              False   
              7               2                              False   
72            0               2                              False   

                                                   consonant_diacritic  \
grapheme_root vowel_diacritic consonant_diacritic                        
64            3               2                                  False   
              7               2                                  False   
72            0               2                                  False   

                                                   grapheme  
grapheme_root vowel_diacritic consonant_diacritic            
64            3               2                        True  
              7               2                        True  
72            0               2                        True

In [17]:
multilabled_graphemes = {
    "64-3-2": dataset.query("grapheme_root == 64 & vowel_diacritic == 3 & consonant_diacritic == 2")['grapheme'].unique().tolist(),
    "64-7-2": dataset.query("grapheme_root == 64 & vowel_diacritic == 7 & consonant_diacritic == 2")['grapheme'].unique().tolist(),
    "72-0-2": dataset.query("grapheme_root == 72 & vowel_diacritic == 0 & consonant_diacritic == 2")['grapheme'].unique().tolist(),
}
multilabled_graphemes

{'64-3-2': ['র্তী', 'র্ত্রী'],
 '64-7-2': ['র্তে', 'র্ত্রে'],
 '72-0-2': ['র্দ্র', 'র্দ']}

As discovered by @ren4yu[[1](https://www.kaggle.com/c/bengaliai-cv19/discussion/133735#763562)] the unicode itself is encoded using the root/vowel/consonant diacritics as a multibyte string, and the unicode consortium have implemented multiple renderings of the same grapheme by allowing duplicate diacritics within the unicode. 

This potentually opens up another datasource for investigation, which is to explore the full range of diacritic combinations within the unicode specification.

The paper [Fonts-2-Handwriting: A Seed-Augment-Train framework for universal digit classification](https://arxiv.org/pdf/1905.08633.pdf) also makes the suggestion that it may be possible to generate synethetic data for handwriting recognition by rendering each of the unicode graphemes using various Bengali fonts

This simply counts how many times each of these multi-keyed unicode graphemes is listed in the database

In [18]:
dataset[ dataset['grapheme'].isin(multilabled_grapheme_list) ].groupby(['grapheme']).count()['image_id'].to_dict()


NameError: name 'multilabled_grapheme_list' is not defined